In [13]:
import pandas as pd
from datetime import datetime, timedelta

## Editable parameters

In [18]:
hedge_intervals = 5 # hedge interval in minutes
weight = [0.5,0.5] # weights for the two assets
stocks = ["BAJAJ-AUTO", "BAJFINANCE"] # Stock symbols
start_date = "20240101"

## Load data

In [15]:
# Function to read a CSV file for a given stock and date
def read_csv_file(stock, date):
    dt = datetime.strptime(date, "%Y%m%d")
    year = dt.year
    month = dt.month

    def get_last_thursday(y, m):
        if m == 12:
            last_day = datetime(y + 1, 1, 1) - timedelta(days=1)
        else:
            last_day = datetime(y, m + 1, 1) - timedelta(days=1)
        offset = (last_day.weekday() - 3) % 7  # Thursday is 3
        return last_day - timedelta(days=offset)

    last_thurs_current = get_last_thursday(year, month)

    # If last Thursday of current month is before the input date,
    # get last Thursday of next month
    if last_thurs_current < dt:
        # increment month/year for next month
        if month == 12:
            next_month = 1
            next_year = year + 1
        else:
            next_month = month + 1
            next_year = year
        last_thurs = get_last_thursday(next_year, next_month)
    else:
        last_thurs = last_thurs_current

    last_thurs_str = last_thurs.strftime("%Y%m%d")

    file_path = f"{stock}/{year}/{year}_new/{stock}_{date}_{last_thurs_str}_Intraday_Preprocessed.csv"

    try:
        return pd.read_csv(file_path)
    except FileNotFoundError:
        print(f"File not found: {file_path}")
        return None

In [16]:
S_1 = read_csv_file(stocks[0], start_date)
S_2 = read_csv_file(stocks[1], start_date)

In [9]:
S_1.columns

Index(['Date Time', 'ExchToken', 'BidPrice', 'BidQty', 'AskPrice', 'AskQty',
       'TTq', 'LTP', 'TotalTradedPrice', 'Instrument', 'ExpiryDate',
       'ExpiryTime', 'Strike', 'Type', 'ExpiryDateTime', 'Spot',
       'Time_to_expire', 'Delta', 'Theta', 'Gamma', 'Vega', 'Sigma',
       'bid_ask_spread', 'mid_price', 'Intrinsic_value', 'bid_ask_move',
       'price_problem', 'is_tradable', 'bid_plus', 'ask_minus'],
      dtype='object')

In [ ]:
def get_delta_at_time(df, time, spot, strike):
    # Get CE and PE rows at time and strike
    ce_row = df[(df['Date Time'] == time) & (df['Type'] == 'CE') & (df['Strike'] == strike)]
    pe_row = df[(df['Date Time'] == time) & (df['Type'] == 'PE') & (df['Strike'] == strike)]

    if ce_row.empty or pe_row.empty:
        return None

    delta_ce = ce_row.iloc[0]['Delta']
    delta_pe = pe_row.iloc[0]['Delta']
    return delta_ce + delta_pe

In [ ]:
def simulate_delta_hedging(option_dfs, weights, interval_minutes=5):
    timestamps = pd.date_range("2024-01-01 09:20", "2024-01-01 15:15", freq=f'{interval_minutes}min')
    
    hedge_log = []

    for t in timestamps:
        deltas = []
        hedge_per_stock = []

        for i, df in enumerate(option_dfs):
            weight = weights[i]
            df_t = df[df['Date Time'] == t]

            if df_t.empty:
                deltas.append(0)
                hedge_per_stock.append(0)
                continue

            spot = df_t['Spot'].iloc[0]
            strikes = df_t['Strike'].unique()
            atm_strike = min(strikes, key=lambda x: abs(x - spot))

            d_j = get_delta_at_time(df, t, spot, atm_strike)
            if d_j is None:
                deltas.append(0)
                hedge_per_stock.append(0)
                continue

            wj_dj = weight * d_j
            deltas.append(wj_dj)
            hedge_per_stock.append(-wj_dj * spot)  # Hedge with spot

        total_delta = sum(deltas)
        hedge_log.append({
            'Time': t,
            'Deltas': deltas,
            'OD^t': total_delta,
            'Spot Hedge Values': hedge_per_stock,
            'Total Hedge (Spot)': sum(hedge_per_stock)
        })

    return pd.DataFrame(hedge_log)

In [20]:
S_1_straddles = get_atm_straddles_exact(S_1, "09:20", "15:15", 5)
S_2_straddles = get_atm_straddles_exact(S_2, "09:20", "15:15", 5)

In [21]:
print("S_1 ATM Straddles:\n", S_1_straddles.head())
# print("S_2 ATM Straddles:\n", S_2_straddles.head())

S_1 ATM Straddles:
             Date Time    Spot  Strike  Call Price  Put Price  Straddle Price
0 2024-01-01 09:20:00  6797.0  6800.0     20565.0    20830.0         41395.0
1 2024-01-01 09:25:00  6778.0  6800.0     19045.0    21350.0         40395.0
2 2024-01-01 09:30:00  6793.0  6800.0     19985.0    20800.0         40785.0
3 2024-01-01 09:35:00  6804.0  6800.0     20520.0    19910.0         40430.0
4 2024-01-01 09:40:00  6796.0  6800.0     19780.0    19900.0         39680.0


In [ ]:
import pandas as pd
from datetime import datetime

def compute_portfolio_pnl(option_dfs, weights, lot_sizes, interval_minutes=5, 
                           start_time_str="09:20", end_time_str="15:15", 
                           use_mid_price=True):
    timestamps = pd.date_range("2024-01-01 " + start_time_str, 
                               "2024-01-01 " + end_time_str, 
                               freq=f'{interval_minutes}min')

    portfolio_pnl_records = []
    initial_costs = []
    previous_hedges = [0] * len(option_dfs)
    hedge_cashflows = [0] * len(option_dfs)

    for i, df in enumerate(option_dfs):
        df['Date Time'] = pd.to_datetime(df['Date Time'])

    for t in timestamps:
        stock_pnls = []

        for i, df in enumerate(option_dfs):
            weight = weights[i]
            lot_size = lot_sizes[i]

            df_t = df[df['Date Time'] == t]
            if df_t.empty:
                stock_pnls.append({'Option PnL': 0, 'Hedge PnL': 0})
                continue

            spot = df_t['Spot'].iloc[0]
            strikes = df_t['Strike'].unique()
            atm_strike = min(strikes, key=lambda x: abs(x - spot))

            ce = df_t[(df_t['Strike'] == atm_strike) & (df_t['Type'] == 'CE')]
            pe = df_t[(df_t['Strike'] == atm_strike) & (df_t['Type'] == 'PE')]

            if ce.empty or pe.empty:
                stock_pnls.append({'Option PnL': 0, 'Hedge PnL': 0})
                continue

            if use_mid_price:
                ce_price = (ce['BidPrice'].iloc[0] + ce['AskPrice'].iloc[0]) / 2
                pe_price = (pe['BidPrice'].iloc[0] + pe['AskPrice'].iloc[0]) / 2
            else:
                ce_price = ce['BidPrice'].iloc[0]
                pe_price = pe['BidPrice'].iloc[0]

            delta = ce['Delta'].iloc[0] + pe['Delta'].iloc[0]

            if t == timestamps[0]:
                # Initial setup
                ce_ask = ce['AskPrice'].iloc[0]
                pe_ask = pe['AskPrice'].iloc[0]
                initial_costs.append((ce_ask + pe_ask) * lot_size)
                hedge_value = delta * spot * lot_size
                previous_hedges[i] = hedge_value
                hedge_cashflows[i] -= hedge_value
                stock_pnls.append({'Option PnL': 0, 'Hedge PnL': 0})
                continue

            current_value = (ce_price + pe_price) * lot_size
            option_pnl = current_value - initial_costs[i]

            hedge_value = delta * spot * lot_size
            hedge_trade = hedge_value - previous_hedges[i]
            hedge_cashflows[i] -= hedge_trade
            hedge_pnl = hedge_cashflows[i] + hedge_value
            previous_hedges[i] = hedge_value

            stock_pnls.append({'Option PnL': option_pnl, 'Hedge PnL': hedge_pnl})

        total_option_pnl = sum(weights[i] * stock_pnls[i]['Option PnL'] for i in range(len(option_dfs)))
        total_hedge_pnl = sum(weights[i] * stock_pnls[i]['Hedge PnL'] for i in range(len(option_dfs)))
        total_pnl = total_option_pnl + total_hedge_pnl

        portfolio_pnl_records.append({
            'Time': t,
            'Option PnL': total_option_pnl,
            'Hedge PnL': total_hedge_pnl,
            'Total PnL': total_pnl
        })

    return pd.DataFrame(portfolio_pnl_records)


Recall the project where we built a delta-hedged ATM straddle portfolio using single stock options. Here's the full context:

- We load preprocessed intraday options data from CSV files located at:
  {STOCK}/{YYYY}/{YYYY}_new/{STOCK}_{STARTDATE}_{LAST_THURSDAY}_Intraday_Preprocessed.csv
  where STARTDATE is a Monday like 20240101, and LAST_THURSDAY is the last Thursday of that month, or of next month if already passed.

- The CSV contains CE rows first, followed by PE rows, with timestamp-aligned data.
- Relevant columns: 'Date Time', 'Strike', 'Type', 'AskPrice', 'BidPrice', 'Delta', 'Spot', 'Sigma', etc.

Strategy:

1. Build a portfolio of ATM straddles (1 CE + 1 PE) on multiple stocks.
2. Buy both legs at AskPrice (customer side) using lot sizes per stock.
3. Hedge delta every h minutes using spot (not futures).
4. At each step:
   - Find ATM strike using Spot
   - Get Delta of CE and PE, sum for total option delta
   - Adjust hedge by buying/selling -delta × spot × lot_size
   - Track hedge cashflows and position delta
5. Use mid-price or bid-price (configurable) to mark straddle value at each step.
6. Compute:
   - Option PnL = (Mark-to-market - Initial Cost) × lot_size
   - Hedge PnL = cashflows + MTM of current hedge
   - Portfolio PnL = weighted sum over all stocks

Functions already implemented:
- last_thursday logic
- load CSVs dynamically
- extract ATM straddles at each interval
- delta hedge simulation
- total PnL calculation per interval

Future requests may include:
- Adding slippage/transaction cost
- Multi-day simulation
- Export/plot PnL
